In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/HME/Temporal-Transformer

# requirements installation
!pip install einops
!pip install PyPI
!pip install timm
!pip install import-ipynb
# !pip install ipynb

import import_ipynb
import torch.optim as optim
from main.arguments import parse_args
from main.plot_training_curve import train_plot, test_plot
import numpy as np
import os

args = parse_args()
print(args)

%run main/data_generator.ipynb

from model import *
# from model import VideoTransformer

receptive_field = 8
# initialize the model
model_pos_train = VideoTransformer(num_frame=receptive_field, in_chans = 82, embed_dim_ratio = 16, num_heads=8, mlp_ratio=2., qkv_bias=True, qk_scale=None, drop_path_rate=0.1)
model_pos = VideoTransformer(num_frame=receptive_field, in_chans = 82, embed_dim_ratio = 16, num_heads=8, mlp_ratio=2., qkv_bias=True, qk_scale=None, drop_path_rate=0.1)


# convert the data structure to cuda
if torch.cuda.is_available():
  model_pos_train = nn.DataParallel(model_pos_train)
  model_pos_train = model_pos_train.cuda()
  model_pos = nn.DataParallel(model_pos)
  model_pos = model_pos.cuda()

# resume
# load the best epoch
args.resume = 'embed_dim16/best_epoch.bin'   # change this argument to resume the train or start a new training

if args.resume or args.evaluate:
  chk_filename = os.path.join(args.checkpoint, args.resume if args.resume else args.evaluate)      # check point filename
  print('Loading Checkpoint', chk_filename)
  checkpoint = torch.load(chk_filename, map_location=lambda storage, loc: storage)
  model_pos_train.load_state_dict(checkpoint['model_pos'], strict=False)
  model_pos.load_state_dict(checkpoint['model_pos'], strict=False)

%run main/transformation.ipynb



Mounted at /content/drive
/content/drive/MyDrive/HME/Temporal-Transformer
  Created wheel for PyPI: filename=pypi-2.1-py3-none-any.whl size=1354 sha256=361c418c381a0e94dbd4284125d549fb5c55f3cac49176f292134d208b7c9868
  Stored in directory: /root/.cache/pip/wheels/9d/58/40/27b525c0b051491cabddd0157355cd3365dfafe2e83618baa6
Successfully built PyPI
     |████████████████████████████████| 376 kB 13.1 MB/s 
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=a19435287dbba8a7e412348f5d9fa41d52ae99893b583ef82bc311a57fcb0a76
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb
importing Jupyter notebook from /content/drive/MyDrive/HME/Temporal-Transformer/main/arguments.ipynb
importing Jupyter notebook from /content/drive/MyDrive/HME/Temporal-Transformer/main/plot_training_curve.ipynb
Namespace(batch_size=512, checkpoint='checkpoint', checkpoint_frequency=10, data_a

In [3]:
# visualization
import os

hm_path = '/content/drive/MyDrive/HME/H36M Data/Cropped Images/HMR Outputs/Structure 2_gt'

subject = 'S6'
# video_name = 'Discussion 1.54138969' + '_resultsgt.npy'
video_name = 'Sitting 1.58860488' + '_resultsgt.npy'
# Directions 1.58860488_resultsgt.npy
# Discussion 1.54138969, Greeting 1.58860488, \Sitting 1.58860488

subject_folder = os.path.join(hm_path, subject)
video_path = os.path.join(subject_folder, video_name)

dataloader = Dataloader_zzx_video(video_path, receptive_field=9)

for sample, central, central_name in dataloader.generator():
    print(central_name)


with torch.no_grad():
      error = 0
      model_pos.load_state_dict(model_pos_train.state_dict(), strict=False)
      model_pos.eval()
        # change the args definition in argumentation
      if torch.cuda.is_available():
          central = central.cuda()
          sample = sample.cuda()
          # print('yes')
          predict = model_pos(sample)

print(predict.shape)
# the shape is correct

shape_and_pose = predict.cpu().detach().numpy()

central = central.cpu().detach().numpy()

# convert torch.cuda to numpy
## convert the predicted pose_and_shape to vertices
shape_and_pose = shape_and_pose.squeeze()
central = central.squeeze()

vertices, joints_3d, shapes = smpl2vertices(shape_and_pose)
vertices_gt, joints_3d_gt, shapes_gt = smpl2vertices(central)

vertices = vertices.numpy()
vertices_gt = vertices_gt.numpy()
joints_3d = joints_3d.numpy()
joints_3d_gt = joints_3d_gt.numpy()

print(vertices.shape)

# vertices = vertices.squeeze()
# vertices_gt = vertices_gt.squeeze()

# np.save('vertices'+subject+'.'+video_name, vertices)
# np.save('vertices_gt', vertices_gt)
# print(central_name)

91
dividing data......
[array([['S6', 'Sitting 1.58860488', 'frame_1.jpg']], dtype='<U18'), array([['S6', 'Sitting 1.58860488', 'frame_21.jpg']], dtype='<U18'), array([['S6', 'Sitting 1.58860488', 'frame_41.jpg']], dtype='<U18'), array([['S6', 'Sitting 1.58860488', 'frame_61.jpg']], dtype='<U18'), array([['S6', 'Sitting 1.58860488', 'frame_81.jpg']], dtype='<U18'), array([['S6', 'Sitting 1.58860488', 'frame_101.jpg']], dtype='<U18'), array([['S6', 'Sitting 1.58860488', 'frame_121.jpg']], dtype='<U18'), array([['S6', 'Sitting 1.58860488', 'frame_141.jpg']], dtype='<U18'), array([['S6', 'Sitting 1.58860488', 'frame_161.jpg']], dtype='<U18'), array([['S6', 'Sitting 1.58860488', 'frame_181.jpg']], dtype='<U18'), array([['S6', 'Sitting 1.58860488', 'frame_201.jpg']], dtype='<U18'), array([['S6', 'Sitting 1.58860488', 'frame_221.jpg']], dtype='<U18'), array([['S6', 'Sitting 1.58860488', 'frame_241.jpg']], dtype='<U18'), array([['S6', 'Sitting 1.58860488', 'frame_261.jpg']], dtype='<U18'), ar

In [4]:
names = central_name[0]

for i in central_name[1:]:
    names = np.concatenate((names, i), axis = 0)

print(names.shape)

(91, 3)


In [ ]:
result = {}
result['vertices'] = vertices
result['vertices_gt'] = vertices_gt
result['name'] = names

In [6]:
result = {}
result['vertices'] = vertices
result['vertices_gt'] = vertices_gt
result['name'] = names
result['3d'] = joints_3d
result['gt_3d'] = joints_3d_gt

In [7]:
np.save('vertices.' + subject+'.'+video_name[:-4], result)

In [ ]:
print(result['name'].shape)

(91, 3)


In [ ]:
result['vertices'][10]

array([[ 0.03139038, -0.85753727,  0.2146033 ],
       [ 0.03440664, -0.8404715 ,  0.21313328],
       [ 0.02165426, -0.8434707 ,  0.20593497],
       ...,
       [ 0.13221337, -0.84310985,  0.08192003],
       [ 0.13295452, -0.8440393 ,  0.08434393],
       [ 0.13558565, -0.8399643 ,  0.08503029]], dtype=float32)